In [27]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dropout, Flatten, Dense, Conv1D, Reshape, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam 
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import clone_model

In [6]:
data = pd.read_csv(r"C:\Users\Yash Vikram Singh\Downloads\emotions.csv\emotions.csv")

In [7]:
import chardet

with open(r"C:\Users\Yash Vikram Singh\Downloads\emotions.csv\emotions.csv", "rb") as file:
    result = chardet.detect(file.read())
    print(result["encoding"])

ascii


In [8]:
# import kagglehub
# kagglehub.dataset_download('birdy654/eeg-brainwave-dataset-feeling-emotions', path='emotions.csv')

In [9]:
print(data.columns)

Index(['# mean_0_a', 'mean_1_a', 'mean_2_a', 'mean_3_a', 'mean_4_a',
       'mean_d_0_a', 'mean_d_1_a', 'mean_d_2_a', 'mean_d_3_a', 'mean_d_4_a',
       ...
       'fft_741_b', 'fft_742_b', 'fft_743_b', 'fft_744_b', 'fft_745_b',
       'fft_746_b', 'fft_747_b', 'fft_748_b', 'fft_749_b', 'label'],
      dtype='object', length=2549)


In [10]:
label_mappping = {'NEGATIVE':0, 'NEUTRAL':1, 'POSITIVE':2}
data['label'] = data['label'].map(label_mappping)

In [11]:
data.head()

,# mean_0_a,mean_1_a,mean_2_a,mean_3_a,mean_4_a,mean_d_0_a,mean_d_1_a,mean_d_2_a,mean_d_3_a,mean_d_4_a,...,fft_741_b,fft_742_b,fft_743_b,fft_744_b,fft_745_b,fft_746_b,fft_747_b,fft_748_b,fft_749_b,label
0,4.62,30.3,-356.0,15.6,26.3,1.070,0.411,-15.70,2.06,3.15,...,23.5,20.3,20.3,23.5,-215.0,280.00,-162.00,-162.00,280.00,0
1,28.80,33.1,32.0,25.8,22.8,6.550,1.680,2.88,3.83,-4.82,...,-23.3,-21.8,-21.8,-23.3,182.0,2.57,-31.60,-31.60,2.57,1
2,8.90,29.4,-416.0,16.7,23.7,79.900,3.360,90.20,89.90,2.03,...,462.0,-233.0,-233.0,462.0,-267.0,281.00,-148.00,-148.00,281.00,2
3,14.90,31.6,-143.0,19.8,24.3,-0.584,-0.284,8.82,2.30,-1.97,...,299.0,-243.0,-243.0,299.0,132.0,-12.40,9.53,9.53,-12.40,2
4,28.30,31.3,45.2,27.3,24.5,34.800,-5.790,3.06,41.40,5.52,...,12.0,38.1,38.1,12.0,119.0,-17.60,23.90,23.90,-17.60,1


In [12]:
X = data.iloc[:, :-1].values
y = pd.get_dummies(data['label']).values


In [13]:
X_train, X_val, Y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state=11)

In [14]:
X_train_lstm = np.expand_dims(X_train, axis=-1)
X_val_lstm = np.expand_dims(X_val, axis=-1)

In [15]:
i_lstm = Input(shape=(X_train.shape[1], 1))
x_lstm = LSTM(256, return_sequences=True)(i_lstm)
x_lstm = Dropout (0.6)(x_lstm)
x_lstm = Flatten()(x_lstm)
y_lstm = Dense(3, activation='softmax')(x_lstm)
model_lstm = Model(i_lstm, y_lstm)

In [16]:
adam_lstm = Adam(learning_rate=0.000001)
es = EarlyStopping(monitor = 'val_accuracy', mode = 'max', patience = 20, restore_best_weights=True)

In [17]:
model_lstm.compile(optimizer=adam_lstm,loss='categorical_crossentropy',metrics=['accuracy'])

In [24]:
lstm_h = model_lstm.fit(X_train_lstm, Y_train,
                        batch_size=512,
                        validation_data=(X_val_lstm, y_val),
                        epochs=1,
                        callbacks=[es])

4/4 [==============================] - 497s 122s/step - loss: 1.0983 - accuracy: 0.4258 - val_loss: 0.9144 - val_accuracy: 0.6557


In [18]:
input_shape_capsule = (X_train.shape[1], 1)
input_layer_capsule = Input(shape=input_shape_capsule)

In [19]:
conv1_params={
    "filters": 256,
    "kernel_size": 11,
    "strides": 1,
    "padding":"valid",
    "activation": "softmax"
}

In [20]:
primary_caps = Conv1D(**conv1_params)(input_layer_capsule)
primary_caps_reshaped = Reshape((-1, 256))(primary_caps)

In [21]:
num_capsule = 5
dim_capsule = 16
capsule_layer = Dense(num_capsule * dim_capsule, activation = "relu")(primary_caps_reshaped)
capsule_layer_flattened = Flatten()(capsule_layer)

In [22]:
def squash(x, axis=-1):
    squared_norm = tf.reduce_sum(tf.square(x), axis, keepdims=True)
    scale = squared_norm / (1+squared_norm)
    return scale*x

In [23]:
capsule_layer_squashed = Lambda(squash)(capsule_layer_flattened)
output_layer_capsule = Dense(3, activation='softmax')(capsule_layer_squashed)

In [24]:
model_capsule = Model(inputs=input_layer_capsule, outputs=output_layer_capsule)
model_capsule.compile(optimizer="adam",
                    loss="categorical_crossentropy",
                    metrics=['accuracy'])

## Applying Particle Swarn Optimization

In [25]:
num_particles = 5
max_iterations = 5
w = 0.5  
c1 = 1.5  
c2 = 1.5 

In [29]:
def flatten_weights(model):
    """Flatten the weights of a Keras model into a 1D array."""
    flattened_weights = []
    for layer in model.layers:
        layer_weights = layer.get_weights()
        if layer_weights:
            flattened_weights.extend([w.flatten() for w in layer_weights])
    return np.concatenate(flattened_weights)

def reconstruct_weights(flattened_weights, model):
    start_idx = 0
    for layer in model.layers:
        layer_weights = layer.get_weights()
        if layer_weights: 
            weight_shapes = [w.shape for w in layer_weights]
            new_weights = [flattened_weights[start_idx:start_idx + np.prod(shape)].reshape(shape)
                           for shape in weight_shapes]
            layer.set_weights(new_weights)
            start_idx += sum([np.prod(shape) for shape in weight_shapes])

def initialize_particles_in_chunks(num_particles, weight_dim, chunk_size=100000):
    """Initialize particles and velocities in chunks to save memory."""
    particles = np.zeros((num_particles, weight_dim), dtype=np.float32)
    velocities = np.zeros((num_particles, weight_dim), dtype=np.float32)
    for start in range(0, weight_dim, chunk_size):
        end = min(start + chunk_size, weight_dim)
        particles[:, start:end] = np.random.uniform(-1, 1, (num_particles, end - start))
        velocities[:, start:end] = np.random.uniform(-0.1, 0.1, (num_particles, end - start))
    return particles, velocities

def fitness_function(weights, model, X_val, y_val):
    """Evaluate the fitness of the weights using validation loss."""
    reconstruct_weights(weights, model)
    loss, accuracy = model.evaluate(X_val, y_val, verbose=0)
    return loss 

def optimize_with_pso_reduced_memory(model, X_train, Y_train, X_val, y_val, num_particles=5, max_iterations=1, w=0.5, c1=1.5, c2=1.5):
    model_clone = clone_model(model)
    model_clone.set_weights(model.get_weights())
    model_clone.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    initial_weights = flatten_weights(model_clone).astype(np.float32)
    weight_dim = len(initial_weights)

    particles, velocities = initialize_particles_in_chunks(num_particles, weight_dim)

    pbest = particles.copy()
    pbest_fitness = np.array([fitness_function(p, model_clone, X_val, y_val) for p in particles])
    gbest = particles[np.argmin(pbest_fitness)]
    gbest_fitness = min(pbest_fitness)

    for iteration in range(max_iterations):
        print(f"Iteration {iteration + 1}/{max_iterations} - Best Fitness: {gbest_fitness}")

        for i in range(num_particles):
            r1, r2 = np.random.rand(weight_dim).astype(np.float32), np.random.rand(weight_dim).astype(np.float32)
            velocities[i] = (w * velocities[i] +
                             c1 * r1 * (pbest[i] - particles[i]) +
                             c2 * r2 * (gbest - particles[i]))

            particles[i] += velocities[i]
            fitness = fitness_function(particles[i], model_clone, X_val, y_val)

            if fitness < pbest_fitness[i]:
                pbest[i] = particles[i]
                pbest_fitness[i] = fitness
                if fitness < gbest_fitness:
                    gbest = particles[i]
                    gbest_fitness = fitness

    reconstruct_weights(gbest, model)
    return model

optimized_lstm = optimize_with_pso_reduced_memory(model_lstm, X_train_lstm, Y_train, X_val_lstm, y_val)

lstm_loss, lstm_accuracy = optimized_lstm.evaluate(X_val_lstm, y_val, verbose=1)
print(f"Optimized LSTM - Loss: {lstm_loss}, Accuracy: {lstm_accuracy}")

optimized_capsule = optimize_with_pso_reduced_memory(model_capsule, X_train_lstm, Y_train, X_val_lstm, y_val)

capsule_loss, capsule_accuracy = optimized_capsule.evaluate(X_val_lstm, y_val)
print(f"Optimized Capsule Network - Loss: {capsule_loss}, Accuracy: {capsule_accuracy}")


Iteration 1/1 - Best Fitness: 129.66722106933594
14/14 [==============================] - 12s 749ms/step - loss: 94.9481 - accuracy: 0.4426
Optimized LSTM - Loss: 94.94805908203125, Accuracy: 0.44262295961380005
Iteration 1/1 - Best Fitness: 91.5803451538086
14/14 [==============================] - 2s 86ms/step - loss: 77.5502 - accuracy: 0.3372
Optimized Capsule Network - Loss: 77.55021667480469, Accuracy: 0.33723652362823486
